Analyse de sentiment : l’objectif est de prédire la note (entre 1 et 0) à partir du texte de la revue d’un film, pour les données issues de la base de données IMDB (50000 revues sur des films, données accessibles sur http://ai.stanford.edu/~amaas/data/sentiment/) ; découper l’ensemble de données en données d’apprentissage et données de test.

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import pandas as pd
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk import stem
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
import string
from sklearn.utils import shuffle
nltk.download(['stopwords','punkt','averaged_perceptron_tagger'])
import warnings
warnings.filterwarnings("ignore")


[nltk_data] Downloading package stopwords to /home/malloc/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/malloc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/malloc/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
data_train = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')

In [3]:
def getdata(data):
    data_d = [[str(i)] for i in  data['Text'].values[0:len(data)]]
    return data_d
    

In [4]:
def getlabel(data):
    label = [[float(i)] for i in  data['Sentiments'].values[0:len(data)]]
    return label

In [5]:
def cleanData(data):
    d=[]
    for sentence in getdata(data):
        stop_words = stopwords.words('english')
        wordwithoustop = ' '.join([word.lower() for word in word_tokenize(sentence[0]) if word not in stop_words + [string.punctuation]])
        word_clean = wordwithoustop.replace("< br / >", "").replace(" '' `` ", "").replace(")", "").replace("(","").replace("``", "") \
        .replace(" , ", ",").replace(".   ", ".").replace(" . ", ".").replace(" 's", "'s").replace(" .",".")\
        .replace("  ", " ").replace(".,", ".").replace(" '' ", " ").replace(" : ", ":").replace(".", " ")\
        .replace(",", " ").replace( ":", " ").replace(";", " ").replace("{", " ").replace("}", " ").replace("--", " ")\
        .replace("'", " ").replace("!"," ").replace("!", " ").replace("/", " ").replace("-", " ").replace("*", " ")
        l = WordNetLemmatizer()
        lstem = [l.lemmatize(i) for i in word_tokenize(word_clean)]
        d.append(" ".join(lstem))
    return pd.DataFrame(d,columns=['Text'])

       

In [6]:
dtrain = cleanData(data_train)
dtrain['label'] = [i[0] for i in getlabel(data_train)]
dtrain= shuffle(dtrain)
dtrain

,Text,label
2532,paul reiser step away standup comedy spotlight...,1.0
15860,ok start little gem ? mutant slug begin take s...,0.0
7957,offbeat slow paced entertaining erotic thrille...,1.0
5619,the girl from missouri arrives new york city k...,1.0
249,i research film prior first viewing part welle...,1.0
...,...,...
13114,bradford dillman play scientist wake one morni...,0.0
23588,we n t television england i walked internet yo...,0.0
6138,i glad i saw film seen director s film past i ...,1.0
3051,all i say n t fall love big little edie watchi...,1.0


In [ ]:
dtest = cleanData(data_test)
dtest['label'] = [i[0] for i in getlabel(data_test)]
dtest= shuffle(dtest)
dtest

In [ ]:
trans_vect = TfidfVectorizer()
X_train = trans_vect.fit_transform(dtrain['Text'])
X_test = trans_vect.transform(dtest['Text'])

In [ ]:
model = MultinomialNB()
model.fit(X_train,dtrain['label'])

In [ ]:
print("Accuration:", accuracy_score(dtest['label'],model.predict(X_test)))